In [1]:
import cv2
import os
import numpy as np
from datasets_wrapper import CacheDataset, cache_read_img
import copy
from pycocotools.coco import COCO
from PIL import Image

In [2]:
coco_image_json = r'/home/epm/YOLOX/datasets/COCO/annotations/instances_val2017.json'
coco_image_dir = r'/home/epm/YOLOX/datasets/COCO/val2017'
des_file = r'/home/epm/YOLOX/YOLOX_outputs/yolox_s_inference'

In [3]:
# c1 = COCODataset(coco_image_dir, coco_image_json,"val2017")

In [138]:
c1.__len__()

70

In [139]:
print(c1._load_coco_annotations())

[(array([[ 34. , 130. ,  61.5, 146. ,   1. ],
       [293. , 271. , 343. , 325. ,   1. ],
       [  0. ,  30. ,  88.5, 166.5,   2. ],
       [ 50. ,  53. , 361. , 350.5,   2. ]]), (416, 416), (416, 416), 'f5d1729aa333b284_jpg.rf.0a241e6c8cfebc2ee5a5a931954e5da7.jpg'), (array([[281. ,  38. , 415.5, 268. ,   2. ],
       [  0. ,   0. , 361. , 383. ,   2. ],
       [162. , 181. , 278. , 224. ,   1. ]]), (416, 416), (416, 416), 'ee047c8ca2cca8a2_jpg.rf.0bc64c938f8b5ab15cd02da914c129cc.jpg'), (array([[201. , 185. , 304.5, 258.5,   1. ],
       [  0. ,   0. , 357. , 415. ,   2. ]]), (416, 416), (416, 416), 'd8daed582e6cce2d_jpg.rf.03386ec76e971c1c9f6a49da8c28ec19.jpg'), (array([[133. , 261. , 202. , 284. ,   1. ],
       [ 70. ,  96. , 370. , 342.5,   2. ]]), (416, 416), (416, 416), 'e5edf93b7c9f5edb_jpg.rf.0cd64b1e0edab1f46d3d52eb35614787.jpg'), (array([[162. , 300. , 224. , 348.5,   1. ],
       [  0. ,  36. ,  12. , 173. ,   2. ],
       [ 43. ,  27. , 347.5, 409.5,   2. ],
       [391. ,

In [65]:
from PIL import Image 
data = Image.fromarray(img)
data.save('test.png')

In [75]:
res, img_info, resized_info, file_name= c1.load_anno_from_ids(1)

In [77]:
res_img = Image.fromarray(res)
res_img.show()

In [19]:
c1.load_anno(1)

array([[281. ,  38. , 415.5, 268. ,   2. ],
       [  0. ,   0. , 361. , 383. ,   2. ],
       [162. , 181. , 278. , 224. ,   1. ]])

In [21]:
img = c1.load_image(1)
img = Image.fromarray(img)
img.show()

In [23]:
img, target, img_info, img_id = c1.__getitem__(1)

In [25]:
target = Image.fromarray(target)
target.show()

# Load Json 

In [3]:
import json
coco_image_json = r'/home/epm/YOLOX/datasets/COCO/annotations/instances_val2017.json'
json_file = open(coco_image_json)
j_file = json.load(json_file)

In [4]:
coco = COCO(coco_image_json)
im_ann = coco.loadImgs(1)[0]

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [63]:
coco_anno = j_file['annotations']
print(len(coco_anno))

218


In [66]:
coco_anno = j_file['annotations']
def load_anno(image_id):
    objs = []
    im_ann = coco.loadImgs(image_id)[0]
    anno_ids = coco.getAnnIds(imgIds=[int(image_id)], iscrowd=False)
    # print("+++++", anno_ids)
    annotations = coco.loadAnns(anno_ids)
    for obj in annotations:
        # x1 = np.max((0, obj['bbox'][0]))
        # y1 = np.max((0, obj['bbox'][1]))
        # x2 = np.min((width, x1 + np.max((0, obj['bbox'][2]))))
        # y2 = np.min((height, y1 + np.max((0, obj['bbox'][3]))))
        x1 = obj['bbox'][0]
        y1 = obj['bbox'][1]
        x2 = x1 + obj['bbox'][2]
        y2 = y1 + obj['bbox'][3]
    
        # print(x1, y1, x2, y2)
        if obj['area'] > 0 and x2 >= x1 and y2 >= y1:
            obj['clean_bbox'] = [x1, y1, x2, y2]
            # print(obj['clean_bbox'])
            objs.append(obj)
    return objs
    
for obj in coco_anno:
    # print(obj)
    image_id = obj['image_id']
    objs = load_anno(image_id)
    num_objs = len(objs)
    res = np.zeros((num_objs, 5))
    for ix, obj in enumerate(objs):
        res[ix, 0:4] = obj['clean_bbox']
        res[ix, 4] = obj['category_id']
        # print(res)

In [11]:
from yolox.data.datasets import COCO_CLASSES
cls_name = COCO_CLASSES

## Test one image

In [61]:
from yolox.data.datasets import COCO_CLASSES
import random
temp_img_id = 1
temp_objs = load_anno(temp_img_id)
num_objs = len(temp_objs)
cls = COCO_CLASSES
green = (0, 255, 0) #license_plate
red = (255, 0, 0)  #car
blue = (0, 0, 255) #plate
# res = np.zeros((num_objs, 5))
img_filename = os.path.join(coco_image_dir, coco.loadImgs(temp_img_id)[0]['file_name'])
print(img_filename)
img = cv2.imread(img_filename)
for ix, obj in enumerate(temp_objs):
    print(obj)
    x1, y1, x2, y2 = obj['clean_bbox']
    category = obj['category_id']
    print(cls[category])
    print(x1, y1, x2, y2)
    # res[ix, 0:4] = obj['clean_bbox']
    # res[ix, 4] = obj['category_id']
    if obj['category_id']== 0:
        color = green
    elif obj['category_id']==1:
        color = red
    else:
        color = blue
    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), color, 2)
    cv2.putText(img, cls[category], (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, thickness=1)
cv2.imwrite('test1.png', img)

+++++ [4, 5, 6]
281 38 415.5 268
0 0 361 383
162 181 278 224
/home/epm/YOLOX/datasets/COCO/val2017/ee047c8ca2cca8a2_jpg.rf.0bc64c938f8b5ab15cd02da914c129cc.jpg
{'id': 4, 'image_id': 1, 'category_id': 2, 'bbox': [281, 38, 134.5, 230], 'area': 30935, 'segmentation': [], 'iscrowd': 0, 'clean_bbox': [281, 38, 415.5, 268]}
car
281 38 415.5 268
{'id': 5, 'image_id': 1, 'category_id': 2, 'bbox': [0, 0, 361, 383], 'area': 138263, 'segmentation': [], 'iscrowd': 0, 'clean_bbox': [0, 0, 361, 383]}
car
0 0 361 383
{'id': 6, 'image_id': 1, 'category_id': 1, 'bbox': [162, 181, 116, 43], 'area': 4988, 'segmentation': [], 'iscrowd': 0, 'clean_bbox': [162, 181, 278, 224]}
license
162 181 278 224


True

## Test directory

In [39]:
coco_image_json = r'/home/epm/YOLOX/datasets/COCO/annotations/instances_val2017.json'
coco_image_dir = r'/home/epm/YOLOX/datasets/COCO/val2017'
json_file = open(coco_image_json)
j_file = json.load(json_file)
coco = COCO(coco_image_json)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [40]:
os.mkdir(r'/home/epm/YOLOX/datasets/COCO/val_annotation_1')

In [67]:
coco_anno = j_file['annotations']
val_annotation = r'/home/epm/YOLOX/datasets/COCO/val_annotation_1'
for obj in coco_anno:
    image_id = obj['image_id']
    img_filename = os.path.join(coco_image_dir, coco.loadImgs(image_id)[0]['file_name'])
    base_name = os.path.basename(img_filename)
    # print(base_name)
    img = cv2.imread(img_filename)
    # print(obj)
    image_id = obj['image_id']
    objs = load_anno(image_id)
    num_objs = len(objs)
    res = np.zeros((num_objs, 5))
    for ix, obj in enumerate(objs):
        x1, y1, x2, y2 = obj['clean_bbox']
        # print(x1, y1, x2, y2)
        res[ix, 0:4] = obj['clean_bbox']
        res[ix, 4] = obj['category_id']
        cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,125, 255), 2)
    # cv2.imwrite(r'/home/epm/YOLOX/datasets/COCO/val_annotation/{}'.format(base_name) , img)
    # cv2.imwrite(f'{val_annotation}/'+ base_name , img)
    # print(obj)

In [70]:
image_id = 9
img_filename = os.path.join(coco_image_dir, coco.loadImgs(image_id)[0]['file_name'])
img = cv2.imread(img_filename)
base_name = os.path.basename(img_filename)
# print(base_name)

anno_ids = coco.getAnnIds(imgIds=[int(image_id)], iscrowd=False)
annotations = coco.loadAnns(anno_ids)
for obj in annotations:
    x1 = obj['bbox'][0]
    y1 = obj['bbox'][1]
    x2 = x1 + obj['bbox'][2]
    y2 = y1 + obj['bbox'][3]
    category = obj['category_id']
    class_name = cls[category]
    # if obj['area'] > 0 and x2 >= x1 and y2 >= y1:
    #     obj['clean_bbox'] = [x1, y1, x2, y2]
    #     # print(obj['clean_bbox'])
    #     objs.append(obj)
    cv2.rectangle(img, (int(x1),int(y1)), (int(x2),int(y2)), (255,125, 255), 2)
cv2.imwrite('test1.png', img)
        

True

In [71]:
class ContextManager():
    def __init__(self):
        print('init method called')
         
    def __enter__(self):
        print('enter method called')
        return self
     
    def __exit__(self, exc_type, exc_value, exc_traceback):
        print('exit method called')
 
with ContextManager() as manager:
    print('with statement block')

init method called
enter method called
with statement block
exit method called


In [74]:
with open(coco_image_json, 'r') as file:
    j_file = json.load(file)

In [91]:
from collections import defaultdict
img_anno = defaultdict(list)
for obj in j_file['annotations']:
    img_anno[obj['image_id']].append(obj['id'])

In [92]:
print(img_anno)

defaultdict(<class 'list'>, {0: [0, 1, 2, 3], 1: [4, 5, 6], 2: [7, 8], 3: [9, 10], 4: [11, 12, 13, 14], 5: [15, 16], 6: [17, 18, 19, 20], 7: [21, 22], 8: [23, 24], 9: [25, 26, 27, 28], 10: [29, 30], 11: [31, 32, 33, 34], 12: [35, 36], 13: [37, 38, 39, 40], 14: [41, 42, 43], 15: [44, 45], 16: [46, 47], 17: [48, 49, 50, 51, 52, 53], 18: [54, 55, 56, 57], 19: [58, 59], 20: [60, 61, 62], 21: [63, 64], 22: [65, 66, 67], 23: [68, 69, 70], 24: [71, 72, 73], 25: [74, 75], 26: [76, 77, 78], 27: [79, 80, 81, 82], 28: [83, 84, 85], 29: [86, 87, 88, 89], 30: [90, 91, 92, 93, 94, 95], 31: [96, 97, 98, 99, 100], 32: [101, 102, 103, 104, 105, 106], 33: [107, 108, 109, 110, 111], 34: [112, 113], 35: [114, 115], 36: [116, 117, 118], 37: [119, 120, 121, 122], 38: [123, 124, 125], 39: [126, 127], 40: [128, 129, 130, 131], 41: [132, 133, 134, 135, 136, 137, 138], 42: [139, 140, 141], 43: [142, 143, 144], 44: [145, 146], 45: [147, 148], 46: [149, 150], 47: [151, 152], 48: [153, 154, 155], 49: [156, 157, 15

In [95]:
for ix,value in img_anno.items():
    print(ix, value)

0 [0, 1, 2, 3]
1 [4, 5, 6]
2 [7, 8]
3 [9, 10]
4 [11, 12, 13, 14]
5 [15, 16]
6 [17, 18, 19, 20]
7 [21, 22]
8 [23, 24]
9 [25, 26, 27, 28]
10 [29, 30]
11 [31, 32, 33, 34]
12 [35, 36]
13 [37, 38, 39, 40]
14 [41, 42, 43]
15 [44, 45]
16 [46, 47]
17 [48, 49, 50, 51, 52, 53]
18 [54, 55, 56, 57]
19 [58, 59]
20 [60, 61, 62]
21 [63, 64]
22 [65, 66, 67]
23 [68, 69, 70]
24 [71, 72, 73]
25 [74, 75]
26 [76, 77, 78]
27 [79, 80, 81, 82]
28 [83, 84, 85]
29 [86, 87, 88, 89]
30 [90, 91, 92, 93, 94, 95]
31 [96, 97, 98, 99, 100]
32 [101, 102, 103, 104, 105, 106]
33 [107, 108, 109, 110, 111]
34 [112, 113]
35 [114, 115]
36 [116, 117, 118]
37 [119, 120, 121, 122]
38 [123, 124, 125]
39 [126, 127]
40 [128, 129, 130, 131]
41 [132, 133, 134, 135, 136, 137, 138]
42 [139, 140, 141]
43 [142, 143, 144]
44 [145, 146]
45 [147, 148]
46 [149, 150]
47 [151, 152]
48 [153, 154, 155]
49 [156, 157, 158, 159, 160, 161]
50 [162, 163]
51 [164, 165]
52 [166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176]
53 [177, 178]
54 [179, 